# Прогнозирование стоимости жилья в жилом массиве

**Цель проекта:** создать модель, предсказывающую медианную стоимость дома


**Задачи проекта:** обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году и сделать предсказания на тестовой выборке. Для оценки качества модели необходимо использовать метрики RMSE, MAE и R2.

**Ход исследования:**
 * Шаг 1: Подготовка данных
 * Шаг 2: Обучение моделей
 * Шаг 3: Анализ результатов
 * Шаг 4: Выводы

In [1]:
# импорт необходимых библиотек

import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionSummary

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator
        
RANDOM_SEED = 2022

pd.options.display.float_format = '{:,.3f}'.format

import warnings
warnings.simplefilter("ignore", ResourceWarning)
warnings.simplefilter("ignore", FutureWarning)

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

/home/alina/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## Подготовка данных

### Обзор данных

Данные были собраны в рамках переписи населения в США. Каждая строка содержит агрегированную статистику о жилом массиве. Жилой массив — минимальная географическая единица с населением от 600 до 3000 человек в зависимости от штата. Одна строка в данных содержит статистику в среднем о 1425.5 обитателях жилого массива.

Прочитаем файл с данными.

In [4]:
df_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Отдельно выведем типы данных столбцов таблицы, а также 10 первых строк.

In [5]:
# выведем названия колонок
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']).head(10))
df_housing = df_housing.select("*").orderBy(F.rand())

# выведем первые 10 строк 
df_housing.show(10)

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -118.24|   33.92|              40.0|     1772.0|         369.0|    1122.0|     324.0|       3.2768|           96100.0|      <1H OCEAN|
|  -121.17|   38.76|              14.0|     2028.0|         255.0|     781.0|     251.0|       6.5322|          394000

Согласно документации, в колонках датасета содержатся следующие данные:

- `longitude` — широта;
- `latitude` — долгота;
- `housing_median_age` — медианный возраст жителей жилого массива;
- `total_rooms` — общее количество комнат в домах жилого массива;
- `total_bedrooms` — общее количество спален в домах жилого массива;
- `population` — количество человек, которые проживают в жилом массиве;
- `households` — количество домовладений в жилом массиве;
- `median_income` — медианный доход жителей жилого массива;
- `median_house_value` — медианная стоимость дома в жилом массиве;
- `ocean_proximity` — близость к океану.

В большинстве колонок хранятся количественные данные, кроме одной — `ocean_proximity`. Она хранит категориальные значения.

In [6]:
# выведем базовые статистики
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736383,35.63186143410876,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070055,206855.81690891474,None
2,stddev,2.0035317235025913,2.135952397457118,12.585557612111625,2181.615251582787,421.38507007403194,1132.4621217653396,382.3297528316114,1.899821717945267,115395.615874414,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


### Обработка пропусков

In [6]:
# выведем количество пропущенных значений в каждой колонке
columns = df_housing.columns

for column in columns:
    check_col = F.col(column).isNull()
    print(column, df_housing.filter(check_col).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Пропуски содержатся только в колонке `total_bedrooms` и составляют около 1% данных в этой колонке. Заменим пропуски на медианное значение в этом столбце.

In [7]:
# выведем медианное значение столбца 'total_bedrooms'
median_bedrooms_value = int(df_housing.approxQuantile('total_bedrooms', [0.5], 0)[0])
median_bedrooms_value

435

In [8]:
# заменим пропуски на медианное значение столбца
df_housing = df_housing.na.fill(value=median_bedrooms_value,subset=["total_bedrooms"])
# снова проверим количество пропусков в столбце
df_housing.filter(F.col("total_bedrooms").isNull()).count()

0

**Разделение на выборки**

Разделим данные на обучающую и тестовую выборки в соотношении 3:1

In [9]:
train_data, test_data = df_housing.randomSplit([.75,.25], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

15444 5196


### Обработка категориальных и количественных значений

In [10]:
# разделим колонки на числовые и категориаьные признаки
categorical_cols = ['ocean_proximity']
numerical_cols  = ['housing_median_age',
                   'total_rooms', 'total_bedrooms', 'population',
                   'households', 'median_income'] #  не включаем столбцы с координатами
target = 'median_house_value'

**Трансформация категориальных признаков**

Сначала приведём текстовые категории в числовое представление с помощью трансформера `StringIndexer`:

In [11]:
string_indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols], handleInvalid='keep') #  , handleInvalid = 'keep'
indexer = string_indexer.fit(train_data)

train_data = indexer.transform(train_data)
test_data = indexer.transform(test_data)

cols = [c for c in train_data.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3) 
test_data.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
+---------------+-------------------+
only showing top 3 rows

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
+---------------+-------------------+
only showing top 3 rows



Преобразуем колонку с категориальными значениями техникой *One hot encoding* с помощью трансформера `OneHotEncoder`:

In [12]:
one_hot_encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
encoder = one_hot_encoder.fit(train_data)

train_data = encoder.transform(train_data)
test_data = encoder.transform(test_data)

cols = [c for c in train_data.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3) 
test_data.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Объединим признаки в один вектор

In [13]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
train_data = categorical_assembler.transform(train_data) 
test_data = categorical_assembler.transform(test_data) 

**Трансформация числовых признаков**

Проведём масштабирование числовых признаков

In [14]:
# объединим числовые признаки в один вектор
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)

In [15]:
# проведём масштабирование признаков
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
scaler = standardScaler.fit(train_data)

train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [16]:
# выведем названия столбцов обновлённой таблицы
print(train_data.columns)
print(test_data.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'numerical_features_scaled']
['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'numerical_features_scaled']


In [17]:
# объединим все признаки в один вектор
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

train_data.select(all_features).show(3)
test_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [4.13137070444968...|
|       (4,[2],[1.0])|     [2.86017971846516...|
|       (4,[2],[1.0])|     [4.13137070444968...|
+--------------------+-------------------------+
only showing top 3 rows

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [1.50953929585661...|
|       (4,[2],[1.0])|     [1.35064042260855...|
|       (4,[2],[1.0])|     [4.13137070444968...|
+--------------------+-------------------------+
only showing top 3 rows



## Обучение моделей

Построим две модели линейной регрессии на разных наборах данных:
* все признаки используются;
* только числовые признаки используются

**Обучение модели линейной регрессии на всех признаках**

In [18]:
# обучим модель
lr_all = LinearRegression(labelCol=target, featuresCol='features')

model_all = lr_all.fit(train_data)
# получим предсказания модели на тестовой выборке
predictions_all = model_all.transform(test_data)

**Обучение модели линейной регрессии на числовых признаках**

In [19]:
# обучим модель
lr_num = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

model_num = lr_num.fit(train_data)
# получим предсказания модели на тестовой выборке
predictions_num = model_num.transform(test_data)

## Анализ результатов

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2

In [20]:
test_all_result = model_all.evaluate(test_data)
test_num_result = model_num.evaluate(test_data)

In [21]:
print("Набор данных со всеми признаками (выборка №1):")
print("RMSE = %g" % test_all_result.rootMeanSquaredError)
print("MAE = %g" % test_all_result.meanAbsoluteError)
print("R2 = %g" % test_all_result.r2)

Набор данных со всеми признаками (выборка №1):
RMSE = 69465.1
MAE = 50411.9
R2 = 0.641384


In [22]:
print("Набор данных с числовыми признаками (выборка №2):")
print("RMSE = %g" % test_num_result.rootMeanSquaredError)
print("MAE = %g" % test_num_result.meanAbsoluteError)
print("R2 = %g" % test_num_result.r2)

Набор данных с числовыми признаками (выборка №2):
RMSE = 75333
MAE = 55163.1
R2 = 0.578238


In [23]:
spark.stop()

Для двух наборов данных (все столбцы и только числовые столбцы - выборка №1 и выборка №2 соответственно) рассмотрены три метрики:

* **Корень из среднеквадратичной ошибки**

Метрика показывает корень из среднего квадрата ошибки и неограничена сверху.  
Для выборки №1 значение данной метрики оказалось меньше, чем для выборки №2. Следовательно, согласно данному критерию, модель, обученная на общей выборке (№1), имеет лучшее качество.

* **Средняя абсолютная ошибка**

Этот критерий показывает среднюю сумму отклонений (по модулю) предсказаний модели от настоящих значений из тестовых данных.  
Для выборки №1 значение этой ошибки также оказалось меньше, чем для выборки №2. Также стоит заметить, что в целом значение данной ошибки меньше, чем значение RMSE: это обусловлено тем, что метрика RMSE более чувствительна к выбросам в данных.

* **Коэффициент детерминации**

Данная метрика позволяет интерпретировать значение среднеквадратичной ошибки: случай $R^2 = 1$ соответствует идеальной модели, а случай $R^2 = 0$ - оптимальной константной модели. Чем ближе значение $R^2$ к единице, тем лучше модель.
Модель, обученная на выборке №1, показывает более высокий результат $R^2$ по сравнению с моделью, обученной на выборке №2.


Таким образом, по трём рассмотренным критериям, модель, обученная на всех столбцах (количественных и категориальных) показывает лучший результат.

# Выводы

В рамках данного проекта была разработана модель линейной регрессии, предсказывающая медианную стоимость дома на основе данных о жилье в Калифорнии в 1990 году. Для реализации цели проекта были выполнены следующие задачи:
* произведена подготовка данных, включающая обработку пропущенных значений и трансформацию категориальных (One-Hot Encoding) и количественных признаков (масштабирование).
* построены две модели линейной регрессии на разных наборах данных:
    * набор, содержащий все данные
    * набор, содержащий только числовые данные, без категориальных
* результаты работы двух моделей линейной регрессии были сравнены с помощью метрик RMSE, MAE и $R^2$: по рассмотренным критериям наилучшее качество достигается при обучениии модели на полном наборе данных (количественные и категориальные данные вместе).